In [ ]:
#
# New Open AI 01 model
# 

using SparseArrays
using LinearAlgebra

# local modules
push!(LOAD_PATH, realpath("../src"))
# using lp_constants
# using lp_utils
using LpProblem
using LpStandardFormConverter
using LpReadMPS
# using lp_revised_simplex

mps_folder_path = "../check/problems/mps_files/"

<details>
    <summary> First model </summary>

```julia
function revised_simplex(lp::LPProblem)
    println("Starting Revised Simplex Method")
    # If the problem is minimization, convert it to maximization by negating c
    if lp.is_minimize
        c = -lp.c
    else
        c = lp.c
    end

    # Copy A, b, vars, constraint_types
    A = lp.A
    b = lp.b
    vars = lp.vars
    constraint_types = lp.constraint_types

    # For now, only handle <= constraints
    if any(ct != '<' for ct in constraint_types)
        error("Only <= constraints are currently supported.")
    end

    # All variables are non-negative
    if any(lp.l .!= 0.0) || any(isfinite.(lp.u))
        error("Only non-negative variables with no upper bounds are currently supported.")
    end

    # Convert inequalities to equalities by adding slack variables
    m, n = size(A)
    A_aug = A
    c_aug = c
    vars_aug = vars

    for i in 1:m
        # Add slack variable
        slack_var = "s_$i"
        e = sparse(zeros(m))
        e[i] = 1.0
        A_aug = hcat(A_aug, e)
        c_aug = vcat(c_aug, 0.0)
        vars_aug = vcat(vars_aug, slack_var)
    end

    # Initialize basis with slack variables
    basic_indices = collect(n .+ (1:m))  # Convert to mutable array
    nonbasic_indices = collect(1:n)      # Convert to mutable array

    # Extract submatrices
    B = Matrix(A_aug[:, basic_indices])  # Basis matrix
    c_B = c_aug[basic_indices]

    # Start iterations
    max_iters = 100
    for iter = 1:max_iters
        println("\nIteration $iter")
        println("Current basis variables: ", vars_aug[basic_indices])
        # Compute basic feasible solution
        x_B = B \ b
        println("Current basic feasible solution x_B: ", x_B)

        # Compute simplex multipliers (dual variables)
        λ = c_B' * inv(B)

        # Compute reduced costs for non-basic variables
        reduced_costs = zeros(length(nonbasic_indices))
        for idx in 1:length(nonbasic_indices)
            j = nonbasic_indices[idx]
            a_j = A_aug[:, j]
            reduced_costs[idx] = c_aug[j] - λ * a_j
        end
        println("Reduced costs: ", reduced_costs)

        # Check for optimality
        if lp.is_minimize
            optimal = all(reduced_costs .>= -1e-8)
        else
            optimal = all(reduced_costs .<= 1e-8)
        end

        if optimal
            println("Optimal solution found.")
            # Construct full solution
            x = zeros(length(vars_aug))
            x[basic_indices] = x_B
            obj_value = dot(c_aug, x)
            if lp.is_minimize
                obj_value = -obj_value
            end
            # Return solution
            solution = Dict("Objective" => obj_value,
                            "Variables" => Dict(zip(vars_aug, x)))
            return solution
        end

        # Determine entering variable
        if lp.is_minimize
            entering_idx_rel = findmin(reduced_costs)[2]
        else
            entering_idx_rel = findmax(reduced_costs)[2]
        end
        entering_idx = nonbasic_indices[entering_idx_rel]
        entering_var = vars_aug[entering_idx]
        println("Entering variable: ", entering_var)

        # Compute direction d = B^{-1} * A_j
        a_j = A_aug[:, entering_idx]
        d = B \ a_j
        println("Direction d: ", d)

        # Determine leaving variable
        ratios = [d[i] > 1e-8 ? x_B[i]/d[i] : Inf for i in 1:length(d)]
        theta, leaving_idx_rel = findmin(ratios)
        if isinf(theta)
            error("Problem is unbounded.")
        end
        leaving_idx = basic_indices[leaving_idx_rel]
        leaving_var = vars_aug[leaving_idx]
        println("Leaving variable: ", leaving_var)
        println("Theta: ", theta)

        # Update basic indices
        basic_indices[leaving_idx_rel] = entering_idx
        nonbasic_indices[entering_idx_rel] = leaving_idx

        # Update B and c_B
        B[:, leaving_idx_rel] = A_aug[:, entering_idx]
        c_B[leaving_idx_rel] = c_aug[entering_idx]
    end

    error("Maximum iterations exceeded.")
end
```

</details>

## Refactored model for new LP Stuct

In [ ]:
function revised_simplex(lp::LPProblem)
    println("Starting Revised Simplex Method")

    # If the problem is minimization, convert it to maximization by negating c
    c = lp.is_minimize ? -lp.c : lp.c

    # Extract necessary components from the LPProblem struct
    A = lp.A
    b = lp.b
    vars = lp.vars
    constraint_types = lp.constraint_types
    l = lp.l
    u = lp.u

    # For now, only handle <= constraints
    if any(ct != 'L' for ct in constraint_types)
        error("Only <= constraints are currently supported.")
    end

    # Ensure all variables are non-negative and have no upper bounds
    if any(l .!= 0.0) || any(isfinite.(u))
        error("Only non-negative variables with no upper bounds are currently supported.")
    end

    # Convert inequalities to equalities by adding slack variables
    m, n = size(A)
    A_aug = A
    c_aug = c
    vars_aug = vars

    for i in 1:m
        # Add slack variable
        slack_var = "s_$i"
        e = sparse(zeros(m))
        e[i] = 1.0
        A_aug = hcat(A_aug, e)
        c_aug = vcat(c_aug, 0.0)
        vars_aug = vcat(vars_aug, slack_var)
    end

    # Initialize basis with slack variables
    basic_indices = collect(n .+ (1:m))  # Basic variables indices
    nonbasic_indices = collect(1:n)      # Non-basic variables indices

    # Extract basis matrix
    B = Matrix(A_aug[:, basic_indices])  # Basis matrix
    c_B = c_aug[basic_indices]

    # Start iterations
    max_iters = 100
    for iter in 1:max_iters
        println("\nIteration $iter")
        println("Current basis variables: ", vars_aug[basic_indices])

        # Compute basic feasible solution
        x_B = B \ b
        println("Current basic feasible solution x_B: ", x_B)

        # Compute simplex multipliers (dual variables)
        λ = c_B' * inv(B)

        # Compute reduced costs for non-basic variables
        reduced_costs = zeros(length(nonbasic_indices))
        for idx in 1:length(nonbasic_indices)
            j = nonbasic_indices[idx]
            a_j = A_aug[:, j]
            reduced_costs[idx] = c_aug[j] - λ * a_j
        end
        println("Reduced costs: ", reduced_costs)

        # Check for optimality
        optimal = lp.is_minimize ? all(reduced_costs .>= -1e-8) : all(reduced_costs .<= 1e-8)
        if optimal
            println("Optimal solution found.")
            x = zeros(length(vars_aug))
            x[basic_indices] = x_B
            obj_value = dot(c_aug, x)
            obj_value = lp.is_minimize ? -obj_value : obj_value

            # Return solution with variable names
            solution = Dict("Objective" => obj_value,
                            "Variables" => Dict(zip(vars_aug, x)))
            return solution
        end

        # Determine entering variable
        entering_idx_rel = lp.is_minimize ? findmin(reduced_costs)[2] : findmax(reduced_costs)[2]
        entering_idx = nonbasic_indices[entering_idx_rel]
        entering_var = vars_aug[entering_idx]
        println("Entering variable: ", entering_var)

        # Compute direction d = B^{-1} * A_j
        a_j = A_aug[:, entering_idx]
        d = B \ a_j
        println("Direction d: ", d)

        # Determine leaving variable
        ratios = [d[i] > 1e-8 ? x_B[i] / d[i] : Inf for i in 1:length(d)]
        theta, leaving_idx_rel = findmin(ratios)
        if isinf(theta)
            error("Problem is unbounded.")
        end
        leaving_idx = basic_indices[leaving_idx_rel]
        leaving_var = vars_aug[leaving_idx]
        println("Leaving variable: ", leaving_var)
        println("Theta: ", theta)

        # Update basic indices
        basic_indices[leaving_idx_rel] = entering_idx
        nonbasic_indices[entering_idx_rel] = leaving_idx

        # Update B and c_B
        B[:, leaving_idx_rel] = A_aug[:, entering_idx]
        c_B[leaving_idx_rel] = c_aug[entering_idx]
    end

    error("Maximum iterations exceeded.")
end

## Adding greater than contraints

In [ ]:
function revised_simplex(lp::LPProblem)
    println("Starting Revised Simplex Method")

    # If the problem is minimization, convert it to maximization by negating c
    c = lp.is_minimize ? -lp.c : lp.c

    # Extract necessary components from the LPProblem struct
    A = lp.A
    b = lp.b
    vars = lp.vars
    constraint_types = lp.constraint_types
    l = lp.l
    u = lp.u

    # Ensure all variables are non-negative and have no upper bounds
    if any(l .!= 0.0) || any(isfinite.(u))
        error("Only non-negative variables with no upper bounds are currently supported.")
    end

    # Convert >= constraints to <= by multiplying the row and the RHS by -1
    for i in 1:length(constraint_types)
        if constraint_types[i] == 'G'
            A[i, :] .= -A[i, :]
            b[i] *= -1
            constraint_types[i] = 'L'  # Change the constraint type to <=
        end
    end

    # Convert inequalities to equalities by adding slack or artificial variables
    m, n = size(A)
    A_aug = A
    c_aug = c
    vars_aug = vars
    artificial_vars = []

    for i in 1:m
        if constraint_types[i] == 'L'
            # Add slack variable for <= constraints
            slack_var = "s_$i"
            e = sparse(zeros(m))
            e[i] = 1.0
            A_aug = hcat(A_aug, e)
            c_aug = vcat(c_aug, 0.0)
            vars_aug = vcat(vars_aug, slack_var)
        elseif constraint_types[i] == 'E'
            # Add artificial variable for equality constraints
            art_var = "a_$i"
            e = sparse(zeros(m))
            e[i] = 1.0
            A_aug = hcat(A_aug, e)
            c_aug = vcat(c_aug, 1e6)  # Large penalty in objective
            vars_aug = vcat(vars_aug, art_var)
            push!(artificial_vars, art_var)
        else
            error("Unsupported constraint type: $constraint_types[i]")
        end
    end

    # Phase 1: Solve the auxiliary problem to remove artificial variables
    println("Starting Phase 1: Feasibility Check")

    # Objective for phase 1 (minimize sum of artificial variables)
    c_phase1 = zeros(size(c_aug))
    for var in artificial_vars
        idx = findfirst(==("$(var)"), vars_aug)
        c_phase1[idx] = 1.0
    end

    # Initialize basis with slack variables and artificial variables
    basic_indices = collect(n .+ (1:m))  # Basic variables indices
    nonbasic_indices = collect(1:n)      # Non-basic variables indices

    # Extract basis matrix
    B = Matrix(A_aug[:, basic_indices])  # Basis matrix
    c_B = c_phase1[basic_indices]

    # Start iterations for phase 1
    max_iters = 100
    for iter in 1:max_iters
        println("\nPhase 1 - Iteration $iter")
        println("Current basis variables: ", vars_aug[basic_indices])

        # Compute basic feasible solution
        x_B = B \ b
        println("Current basic feasible solution x_B: ", x_B)

        # Check for feasibility
        if any(x_B .< 0)
            error("Infeasible basic solution found in Phase 1.")
        end

        # Compute simplex multipliers (dual variables)
        λ = c_B' * inv(B)

        # Compute reduced costs for non-basic variables
        reduced_costs = zeros(length(nonbasic_indices))
        for idx in 1:length(nonbasic_indices)
            j = nonbasic_indices[idx]
            a_j = A_aug[:, j]
            reduced_costs[idx] = c_phase1[j] - λ * a_j
        end
        println("Reduced costs: ", reduced_costs)

        # Check for optimality in Phase 1 (all reduced costs >= 0)
        if all(reduced_costs .>= -1e-8)
            println("Feasible solution found in Phase 1.")
            break
        end

        # Determine entering variable
        entering_idx_rel = findmin(reduced_costs)[2]
        entering_idx = nonbasic_indices[entering_idx_rel]
        entering_var = vars_aug[entering_idx]
        println("Entering variable: ", entering_var)

        # Compute direction d = B^{-1} * A_j
        a_j = A_aug[:, entering_idx]
        d = B \ a_j
        println("Direction d: ", d)

        # Determine leaving variable
        ratios = [d[i] > 1e-8 ? x_B[i] / d[i] : Inf for i in 1:length(d)]
        theta, leaving_idx_rel = findmin(ratios)
        if isinf(theta)
            error("Problem is unbounded in Phase 1.")
        end
        leaving_idx = basic_indices[leaving_idx_rel]
        leaving_var = vars_aug[leaving_idx]
        println("Leaving variable: ", leaving_var)
        println("Theta: ", theta)

        # Update basic indices
        basic_indices[leaving_idx_rel] = entering_idx
        nonbasic_indices[entering_idx_rel] = leaving_idx

        # Update B and c_B
        B[:, leaving_idx_rel] = A_aug[:, entering_idx]
        c_B[leaving_idx_rel] = c_phase1[entering_idx]
    end

    # Remove artificial variables and proceed to Phase 2
    println("Starting Phase 2: Solve original problem")
    artificial_var_indices = [findfirst(==("$(var)"), vars_aug) for var in artificial_vars]
    basic_indices = filter(idx -> !(idx in artificial_var_indices), basic_indices)

    # Remove artificial variables from vars_aug and c_aug
    vars_aug = filter(x -> !(x in artificial_vars), vars_aug)
    c_aug = filter(x -> !(x in artificial_vars), c_aug)

    # Reset the objective to the original problem's objective coefficients
    c_B = c_aug[basic_indices]

    # Start iterations for Phase 2 (similar to Phase 1)
    for iter in 1:max_iters
        println("\nPhase 2 - Iteration $iter")
        println("Current basis variables: ", vars_aug[basic_indices])

        # Compute basic feasible solution
        x_B = B \ b
        println("Current basic feasible solution x_B: ", x_B)

        # Compute simplex multipliers (dual variables)
        λ = c_B' * inv(B)

        # Compute reduced costs for non-basic variables
        reduced_costs = zeros(length(nonbasic_indices))
        for idx in 1:length(nonbasic_indices)
            j = nonbasic_indices[idx]
            a_j = A_aug[:, j]
            reduced_costs[idx] = c_aug[j] - λ * a_j
        end
        println("Reduced costs: ", reduced_costs)

        # Check for optimality in Phase 2
        optimal = lp.is_minimize ? all(reduced_costs .>= -1e-8) : all(reduced_costs .<= 1e-8)
        if optimal
            println("Optimal solution found.")
            x = zeros(length(vars_aug))
            x[basic_indices] = x_B
            obj_value = dot(c_aug, x)
            obj_value = lp.is_minimize ? -obj_value : obj_value

            # Return solution with variable names
            solution = Dict("Objective" => obj_value,
                            "Variables" => Dict(zip(vars_aug, x)))
            return solution
        end

        # Determine entering variable
        entering_idx_rel = lp.is_minimize ? findmin(reduced_costs)[2] : findmax(reduced_costs)[2]
        entering_idx = nonbasic_indices[entering_idx_rel]
        entering_var = vars_aug[entering_idx]
        println("Entering variable: ", entering_var)

        # Compute direction d = B^{-1} * A_j
        a_j = A_aug[:, entering_idx]
        d = B \ a_j
        println("Direction d: ", d)

        # Determine leaving variable
        ratios = [d[i] > 1e-8 ? x_B[i] / d[i] : Inf for i in 1:length(d)]
        theta, leaving_idx_rel = findmin(ratios)
        if isinf(theta)
            error("Problem is unbounded in Phase 2.")
        end
        leaving_idx = basic_indices[leaving_idx_rel]
        leaving_var = vars_aug[leaving_idx]
        println("Leaving variable: ", leaving_var)
        println("Theta: ", theta)

        # Update basic indices
        basic_indices[leaving_idx_rel] = entering_idx
        nonbasic_indices[entering_idx_rel] = leaving_idx

        # Update B and c_B
        B[:, leaving_idx_rel] = A_aug[:, entering_idx]
        c_B[leaving_idx_rel] = c_aug[entering_idx]
    end

    error("Maximum iterations exceeded.")
end


## Claude

In [ ]:
function revised_simplex(lp::LPProblem)
    println("Starting Revised Simplex Method")
    # If the problem is minimization, convert it to maximization by negating c
    c = lp.is_minimize ? -lp.c : lp.c
    
    # Extract necessary components from the LPProblem struct
    A = lp.A
    b = lp.b
    vars = lp.vars
    constraint_types = lp.constraint_types
    l = lp.l
    u = lp.u
    
    # Ensure all variables are non-negative and have no upper bounds
    if any(l .!= 0.0) || any(isfinite.(u))
        error("Only non-negative variables with no upper bounds are currently supported.")
    end
    
    # Convert inequalities to equalities by adding slack/surplus variables
    m, n = size(A)
    A_aug = A
    c_aug = c
    vars_aug = vars
    b_aug = b
    
    for i in 1:m
        if constraint_types[i] == 'L'
            # Add slack variable for <= constraint
            slack_var = "s_$i"
            e = sparse(zeros(m))
            e[i] = 1.0
            A_aug = hcat(A_aug, e)
            c_aug = vcat(c_aug, 0.0)
            vars_aug = vcat(vars_aug, slack_var)
        elseif constraint_types[i] == 'G'
            # Add surplus variable for >= constraint
            surplus_var = "s_$i"
            e = sparse(zeros(m))
            e[i] = -1.0
            A_aug = hcat(A_aug, e)
            c_aug = vcat(c_aug, 0.0)
            vars_aug = vcat(vars_aug, surplus_var)
        elseif constraint_types[i] == 'E'
            # No need to add variables for equality constraints
        else
            error("Unsupported constraint type: $(constraint_types[i])")
        end
    end
    
    # Initialize basis
    basic_indices = Int[]
    nonbasic_indices = collect(1:n)
    
    # Add artificial variables for equality constraints and >= constraints
    for i in 1:m
        if constraint_types[i] == 'E' || constraint_types[i] == 'G'
            art_var = "a_$i"
            e = sparse(zeros(m))
            e[i] = 1.0
            A_aug = hcat(A_aug, e)
            c_aug = vcat(c_aug, -1e6)  # Big M method
            vars_aug = vcat(vars_aug, art_var)
            push!(basic_indices, size(A_aug, 2))
        elseif constraint_types[i] == 'L'
            push!(basic_indices, n + i)
        end
    end
    
    nonbasic_indices = setdiff(1:size(A_aug, 2), basic_indices)
    
    # Extract basis matrix
    B = Matrix(A_aug[:, basic_indices])  # Basis matrix
    c_B = c_aug[basic_indices]
    
    # Start iterations
    max_iters = 100
    for iter in 1:max_iters
        println("\nIteration $iter")
        println("Current basis variables: ", vars_aug[basic_indices])
        
        # Compute basic feasible solution
        x_B = B \ b_aug
        println("Current basic feasible solution x_B: ", x_B)
        
        # Compute simplex multipliers (dual variables)
        λ = c_B' * inv(B)
        
        # Compute reduced costs for non-basic variables
        reduced_costs = zeros(length(nonbasic_indices))
        for idx in 1:length(nonbasic_indices)
            j = nonbasic_indices[idx]
            a_j = A_aug[:, j]
            reduced_costs[idx] = c_aug[j] - λ * a_j
        end
        println("Reduced costs: ", reduced_costs)
        
        # Check for optimality
        optimal = lp.is_minimize ? all(reduced_costs .>= -1e-8) : all(reduced_costs .<= 1e-8)
        if optimal
            println("Optimal solution found.")
            x = zeros(length(vars_aug))
            x[basic_indices] = x_B
            obj_value = dot(c_aug, x)
            obj_value = lp.is_minimize ? -obj_value : obj_value
            
            # Return solution with variable names
            solution = Dict("Objective" => obj_value,
                            "Variables" => Dict(zip(vars_aug, x)))
            return solution
        end
        
        # Determine entering variable
        entering_idx_rel = lp.is_minimize ? findmin(reduced_costs)[2] : findmax(reduced_costs)[2]
        entering_idx = nonbasic_indices[entering_idx_rel]
        entering_var = vars_aug[entering_idx]
        println("Entering variable: ", entering_var)
        
        # Compute direction d = B^{-1} * A_j
        a_j = A_aug[:, entering_idx]
        d = B \ a_j
        println("Direction d: ", d)
        
        # Determine leaving variable
        ratios = [d[i] > 1e-8 ? x_B[i] / d[i] : Inf for i in 1:length(d)]
        theta, leaving_idx_rel = findmin(ratios)
        if isinf(theta)
            error("Problem is unbounded.")
        end
        leaving_idx = basic_indices[leaving_idx_rel]
        leaving_var = vars_aug[leaving_idx]
        println("Leaving variable: ", leaving_var)
        println("Theta: ", theta)
        
        # Update basic indices
        basic_indices[leaving_idx_rel] = entering_idx
        nonbasic_indices[entering_idx_rel] = leaving_idx
        
        # Update B and c_B
        B[:, leaving_idx_rel] = A_aug[:, entering_idx]
        c_B[leaving_idx_rel] = c_aug[entering_idx]
    end
    error("Maximum iterations exceeded.")
end

In [ ]:
# Define the problem data
is_minimize = false
c = [3.0, 2.0]
A = sparse([1.0 1.0; 1.0 -1.0; -1.0 1.0; -1.0 -1.0])
b = [4.0, 2.0, -2, -1.0]
constraint_types = ['L', 'L', 'L', 'L']
l = [0.0, 0.0]
u = [Inf, Inf]
vars = ["x1", "x2"]
variable_types = [:Continuous, :Continuous]

# Create LPProblem instance
lp = LPProblem(is_minimize, c, A, b, constraint_types, l, u, vars, variable_types)

# Solve the LP problem
solution = revised_simplex(lp)

# Display the solution
println("Objective Value: ", solution["Objective"])
println("Variable Values: ")
for (var, value) in solution["Variables"]
    println("$var = $value")
end


In [ ]:
mps_filename = mps_folder_path * "ex_9-7.mps"
lp = read_mps_from_file(mps_filename)
print(lp)

In [ ]:
revised_simplex(lp)

### simple problem

In [ ]:
mps_filename = mps_folder_path * "simple.mps"
lp = read_mps_from_file(mps_filename)
print(lp)

In [ ]:
revised_simplex(lp)

### Blend problem

In [ ]:
mps_filename = mps_folder_path * "problem.mps"
lp = read_mps(mps_filename)
revised_simplex(lp)

In [ ]:
mps_filename = mps_folder_path * "test.mps"
lp = read_mps(mps_filename)
revised_simplex(lp)

## Standard form

In [ ]:
# Original problem with variables and constraints
lp = LPProblem(
    false,  # Maximize
    [3.0, 2.0],  # Objective function coefficients
    sparse([1, 2, 1, 3], [1, 1, 2, 2], [1.0, 1.0, 1.0, 1.0], 3, 2),  # Constraint matrix
    [4.0, 2.0, 3.0],  # Right-hand side
    ['L', 'G', 'E'],  # Constraints: <=, >=, =
    [-1.0, 0.0],  # Lower bounds (note the negative lower bound for the first variable)
    [Inf, Inf],  # Upper bounds
    ["x1", "x2"],  # Variable names
    [:Continuous, :Continuous]  # Variable types
)

# Convert to standard form with verbose output
standard_lp = convert_to_standard_form(lp; verbose=true)


## Meta AI and OpenAI o1

In [ ]:
function revised_simplex_method(lp::LPProblem; verbose::Bool = false)::Tuple{Vector{Float64}, Float64}
    if verbose
        println()
        println("#" ^ 80)
        println("Starting Revised Simplex Method with Phase I and Phase II...")
        println("#" ^ 80)
    end

    # Unpack LP problem components
    c = copy(lp.c)
    A = copy(lp.A)
    b = copy(lp.b)
    vars = copy(lp.vars)
    m, n = size(A)  # Number of constraints and variables

    # Ensure that the LP problem is in standard form
    if any(lp.constraint_types .!= 'E')
        error("All constraints must be equalities in standard form.")
    end
    if any(lp.l .!= 0.0)
        error("All variables must have a lower bound of 0 in standard form.")
    end
    if any(isfinite.(lp.u) .& (lp.u .< Inf))
        error("All variables must be unbounded above in standard form.")
    end

    # Check if the initial basic feasible solution is feasible
    # For constraints with negative b_i, multiply the constraint by -1
    for i in 1:m
        if b[i] < 0
            b[i] = -b[i]
            A[i, :] = -A[i, :]
        end
    end

    # Phase I: Introduce artificial variables where needed
    A_phase1 = copy(A)
    c_phase1 = copy(c)
    artificial_vars = Int[]
    num_artificial = 0

    # For each constraint, if the basic variable is negative or zero, add an artificial variable
    for i in 1:m
        num_artificial += 1
        artificial_var_index = n + num_artificial
        # Add column to A_phase1
        A_phase1 = hcat(A_phase1, sparsevec([i], [1.0], m))
        # Update c_phase1
        c_phase1 = vcat(c_phase1, 1.0)
        # Update variable names
        push!(vars, "a_$num_artificial")
        push!(artificial_vars, artificial_var_index)
    end    

    # Update total number of variables
    total_vars = n + num_artificial

    # Initial basic variables are the artificial variables
    basic_indices = collect(n + 1:total_vars)
    nonbasic_indices = collect(1:n)

    # LU factorization of B
    B = A_phase1[:, basic_indices]
    lu_B = lu(Matrix(B))

    # Initial basic feasible solution
    x_B = b
    x_N = zeros(length(nonbasic_indices))

    if verbose
        println("Phase I: Solving for initial basic feasible solution...")
        println("Initial Basic Variables (indices): ", basic_indices)
        println("Initial Non-Basic Variables (indices): ", nonbasic_indices)
        println("Initial Basic Feasible Solution x_B: ", x_B)
        println("-" ^ 80)
    end

    # Phase I Iterations
    while true
        # Compute dual variables
        y = lu_B' \ c_phase1[basic_indices]

        # Compute reduced costs
        N = A_phase1[:, nonbasic_indices]
        reduced_costs = c_phase1[nonbasic_indices] - N' * y

        # Check for optimality
        if all(reduced_costs .>= -1e-10)
            # Optimal solution found for Phase I
            break
        end

        # Determine entering variable
        entering_index_in_N = findmin(reduced_costs)[2]
        entering_var = nonbasic_indices[entering_index_in_N]

        # Compute direction d
        d = lu_B \ Vector(A_phase1[:, entering_var])

        # Determine leaving variable
        positive_d_indices = findall(d .> 1e-10)
        if isempty(positive_d_indices)
            error("Problem is infeasible during Phase I.")
        end

        ratios = x_B[positive_d_indices] ./ d[positive_d_indices]
        min_ratio, pos = findmin(ratios)
        leaving_index_in_B = positive_d_indices[pos]
        leaving_var = basic_indices[leaving_index_in_B]

        # Update indices
        basic_indices[leaving_index_in_B] = entering_var
        nonbasic_indices[entering_index_in_N] = leaving_var

        # Update B and LU factorization
        B = A_phase1[:, basic_indices]
        lu_B = lu(Matrix(B))

        # Update basic solution
        x_B = lu_B \ b
        x_N = zeros(length(nonbasic_indices))
        
            if verbose
            println("Entering Variable Index: ", entering_var)
            println("Leaving Variable Index: ", leaving_var)
            println("Updated Basic Variables (indices): ", basic_indices)
            println("Updated Basic Feasible Solution x_B: ", x_B)
            println("-" ^ 80)
        end
    end

    # Check feasibility
    artificial_values = x_B[findall(in.(basic_indices, artificial_vars))]
    if any(abs.(artificial_values) .> 1e-8)
        error("Original LP problem is infeasible.")
    end

    if verbose
        println("Phase I completed. Feasible solution found.")
        println("-" ^ 80)
    end

    # Phase II: Remove artificial variables and proceed with original objective
    # Remove artificial variables from basis and A
    A = A_phase1[:, 1:n]
    c = copy(lp.c)
    basic_indices = setdiff(basic_indices, artificial_vars)
    total_vars = n
    nonbasic_indices = setdiff(1:total_vars, basic_indices)

    # Update B and N
    B = A[:, basic_indices]
    N = A[:, nonbasic_indices]
    c_B = c[basic_indices]
    c_N = c[nonbasic_indices]

    # LU factorization of B
    lu_B = lu(Matrix(B))

    # Compute initial basic solution
    x_B = lu_B \ b
    x_N = zeros(length(nonbasic_indices))

    if verbose
        println("Phase II: Optimizing original objective function...")
        println("Initial Basic Variables (indices): ", basic_indices)
        println("Initial Non-Basic Variables (indices): ", nonbasic_indices)
        println("Initial Basic Feasible Solution x_B: ", x_B)
        println("-" ^ 80)
    end

    # Phase II Iterations
    iteration = 0
    while true
        iteration += 1

        if verbose
            println("Iteration ", iteration)
        end

        # Compute dual variables
        y = lu_B' \ c_B

        # Compute reduced costs
        reduced_costs = c_N - N' * y

        if verbose
            println("Dual Variables (y): ", y)
            println("Reduced Costs: ", reduced_costs)
        end

        # Check for optimality
        if all(reduced_costs .>= -1e-10)
            # Optimal solution found
            if verbose
                println("Optimal solution found.")
                println("-" ^ 80)
            end
            break
        end

        # Determine entering variable
        entering_index_in_N = findmin(reduced_costs)[2]
        entering_var = nonbasic_indices[entering_index_in_N]

        # Compute direction d
        d = lu_B \ Vector(A[:, entering_var])

        # Determine leaving variable
        positive_d_indices = findall(d .> 1e-10)
        if isempty(positive_d_indices)
            error("Problem is unbounded in the direction of variable $(vars[entering_var]).")
        end

        ratios = x_B[positive_d_indices] ./ d[positive_d_indices]
        min_ratio, pos = findmin(ratios)
        leaving_index_in_B = positive_d_indices[pos]
        leaving_var = basic_indices[leaving_index_in_B]

        # Update indices
        basic_indices[leaving_index_in_B] = entering_var
        nonbasic_indices[entering_index_in_N] = leaving_var

        # Update c_B, c_N
        c_B = c[basic_indices]
        c_N = c[nonbasic_indices]

        # Update B, N
        B = A[:, basic_indices]
        N = A[:, nonbasic_indices]

        # Update LU factorization
        lu_B = lu(Matrix(B))

        # Update basic solution
        x_B = lu_B \ b
        x_N = zeros(length(nonbasic_indices))

        if verbose
            println("Entering Variable Index: ", entering_var, " (", vars[entering_var], ")")
            println("Leaving Variable Index: ", leaving_var, " (", vars[leaving_var], ")")
            println("Updated Basic Variables (indices): ", basic_indices)
            println("Updated Basic Feasible Solution x_B: ", x_B)
            println("-" ^ 80)
        end

        # Check for degeneracy
        if any(x_B .<= 1e-8)
            # Perform degeneracy resolution (e.g., perturbation or lexicographic method)
            # This is a crucial step to ensure the algorithm's correctness
        end
    end

    # Assemble the full solution vector
    x = zeros(total_vars)
    x[basic_indices] = x_B
    x[nonbasic_indices] = x_N
    
    # Compute the optimal value
    optimal_value = c' * x

    if verbose
        println("Optimal Solution x: ", x)
        println("Optimal Objective Value: ", optimal_value)
        println("Revised Simplex Method completed.")
        println("#" ^ 80)
        println()
    end

    return x, optimal_value
end

In [14]:
# # Define the original LP problem
# lp = LPProblem(
#     false,  # Maximize
#     [3.0, 2.0],  # Objective function coefficients
#     sparse([1, 2, 1, 3], [1, 1, 2, 2], [1.0, 1.0, 1.0, 1.0], 3, 2),  # Constraint matrix
#     [4.0, 2.0, 3.0],  # Right-hand side
#     ['L', 'G', 'E'],  # Constraints: <=, >=, =
#     [-1.0, 0.0],  # Lower bounds (negative lower bound for the first variable)
#     [Inf, Inf],  # Upper bounds
#     ["x1", "x2"],  # Variable names
#     [:Continuous, :Continuous]  # Variable types
# )

# # Convert to standard form
# standard_lp = lp_standard_form_converter.convert_to_standard_form(lp)#; verbose=true)

# # Solve the LP problem using the Revised Simplex Method
# optimal_solution, optimal_value = revised_simplex_method(standard_lp; verbose=true)

# # Output the results
# println("Optimal Solution:")
# for i in 1:length(optimal_solution)
#     println("Variable $(standard_lp.vars[i]) = ", optimal_solution[i])
# end
# println("Optimal Objective Value: ", optimal_value)

In [ ]:
mps_filename = mps_folder_path * "simple.mps"
lp = read_mps_from_file(mps_filename)


# Convert to standard form
standard_lp = lp_standard_form_converter.convert_to_standard_form(lp)#; verbose=true)

# Solve the LP problem using the Revised Simplex Method
optimal_solution, optimal_value = revised_simplex_method(standard_lp; verbose=true)

# Output the results
println("Optimal Solution:")
for i in 1:length(optimal_solution)
    println("Variable $(standard_lp.vars[i]) = ", optimal_solution[i])
end
println("Optimal Objective Value: ", optimal_value)

## Using o1-mini to refector the code into subfunctions

In [ ]:
"""
    SimplexResult

Structure to encapsulate the result of the Revised Simplex Method.

# Fields
- `status::Symbol`: Status of the solution (:optimal, :infeasible, :unbounded, :iteration_limit).
- `solution::Dict{String, Float64}`: Mapping of variable names to their optimal values.
- `objective::Float64`: Optimal objective function value.
- `iterations::Int`: Number of iterations performed.
"""
struct SimplexResult
    status::Symbol
    solution::Dict{String, Float64}
    objective::Float64
    iterations::Int
end

In [ ]:
"""
    convert_to_standard_form(lp::LPProblem; verbose::Bool=false)::LPProblem

Converts a general LP problem to standard form by handling inequalities, variable bounds, and adding artificial variables where necessary.

# Arguments
- `lp::LPProblem`: The original linear programming problem.
- `verbose::Bool`: If `true`, prints detailed conversion information.

# Returns
- `LPProblem`: The converted LP problem in standard form.
"""
function convert_to_standard_form_test(lp::LPProblem; verbose::Bool=false)::LPProblem
    if verbose
        println()
        println("#" ^ 80)
        println("~" ^ 80)
        println("Starting conversion to standard form...")
        println("~" ^ 80)
    end

    # Unpack LP problem components
    is_minimize = lp.is_minimize
    c = copy(lp.c)
    A = copy(lp.A)
    b = copy(lp.b)
    constraint_types = copy(lp.constraint_types)
    l = copy(lp.l)
    u = copy(lp.u)
    vars = copy(lp.vars)
    variable_types = copy(lp.variable_types)
    
    m, n = size(A)  # Number of constraints and variables

    # Initialize counts and new structures
    slack_var_count = 0      # Counter for slack variables
    surplus_var_count = 0    # Counter for surplus variables
    artificial_var_count = 0 # Counter for artificial variables (if needed)
    new_vars = String[]      # Names of new variables
    new_variable_types = Symbol[]  # Types of new variables
    new_l = Float64[]        # Lower bounds for new variables
    new_u = Float64[]        # Upper bounds for new variables
    new_A_cols = spzeros(Float64, m, 0)  # Columns to be added to A

    if verbose
        println("Initial problem details:")
        println("Is minimization: ", is_minimize)
        println("Objective function coefficients (c): ", c)
        println("Constraint matrix (A):\n", Matrix(A))
        println("Right-hand side vector (b): ", b)
        println("Constraint types: ", constraint_types)
        println("Variable lower bounds (l): ", l)
        println("Variable upper bounds (u): ", u)
        println("Variable types: ", variable_types)
        println("-" ^ 80)
    end

    # Step 1: Convert maximization to minimization (if necessary)
    if !is_minimize
        c = -c
        is_minimize = true
        if verbose
            println("Converted maximization problem to minimization by negating the objective coefficients.")
            println("New objective function coefficients (c): ", c)
            println("-" ^ 80)
        end
    end

    # Step 2: Adjust variables with negative lower bounds to ensure non-negativity
    for j in 1:n
        if l[j] < 0
            shift_amount = -l[j]
            # Adjust the bounds
            l[j] += shift_amount  # l[j] becomes 0
            if isfinite(u[j])
                u[j] += shift_amount
            end
            # Adjust the corresponding column in A and the RHS vector b
            A_col = A[:, j]
            b .= b .- A_col * shift_amount
            if verbose
                println("Variable $(vars[j]) has negative lower bound.")
                println("Shifting variable by ", shift_amount, " to make it non-negative.")
                println("Updated lower bound l[$j]: ", l[j])
                println("Updated upper bound u[$j]: ", u[j])
                println("Adjusted RHS vector (b): ", b)
                println("-" ^ 80)
            end
        end
    end

    # Step 3: Add slack, surplus, and artificial variables to convert inequalities to equalities
    new_constraint_types = fill('E', m)  # All constraints will be equalities
    for i in 1:m
        ct = constraint_types[i]
        if ct == 'L'
            # Add slack variable
            slack_var_count += 1
            slack_var_name = "s_$slack_var_count"
            slack_column = sparsevec([i], [1.0], m)
            new_A_cols = hcat(new_A_cols, slack_column)
            push!(new_vars, slack_var_name)
            push!(new_variable_types, :Continuous)
            push!(new_l, 0.0)
            push!(new_u, Inf)
            if verbose
                println("Added slack variable '$slack_var_name' to constraint $i (<=).")
            end
        elseif ct == 'G'
            # Add surplus variable
            surplus_var_count += 1
            surplus_var_name = "r_$surplus_var_count"
            surplus_column = sparsevec([i], [-1.0], m)
            new_A_cols = hcat(new_A_cols, surplus_column)
            push!(new_vars, surplus_var_name)
            push!(new_variable_types, :Continuous)
            push!(new_l, 0.0)
            push!(new_u, Inf)
            if verbose
                println("Added surplus variable '$surplus_var_name' to constraint $i (>=).")
            end

            # Add artificial variable for surplus constraint
            artificial_var_count += 1
            artificial_var_name = "a_$artificial_var_count"
            artificial_column = sparsevec([i], [1.0], m)
            new_A_cols = hcat(new_A_cols, artificial_column)
            push!(new_vars, artificial_var_name)
            push!(new_variable_types, :Continuous)
            push!(new_l, 0.0)
            push!(new_u, Inf)
            if verbose
                println("Added artificial variable '$artificial_var_name' to constraint $i (>=).")
            end
        elseif ct == 'E'
            # Equality constraint; add artificial variable
            artificial_var_count += 1
            artificial_var_name = "a_$artificial_var_count"
            artificial_column = sparsevec([i], [1.0], m)
            new_A_cols = hcat(new_A_cols, artificial_column)
            push!(new_vars, artificial_var_name)
            push!(new_variable_types, :Continuous)
            push!(new_l, 0.0)
            push!(new_u, Inf)
            if verbose
                println("Added artificial variable '$artificial_var_name' to constraint $i (=).")
            end
        else
            error("Unknown constraint type: $(ct)")
        end
    end

    # Step 4: Update the constraint matrix A and objective function coefficients c
    A = hcat(A, new_A_cols)
    c = vcat(c, zeros(length(new_vars)))  # Objective coefficients for new variables are zero

    # Step 5: Update variable lists and bounds
    vars = vcat(vars, new_vars)
    variable_types = vcat(variable_types, new_variable_types)
    l = vcat(l, new_l)
    u = vcat(u, new_u)

    # Final verbose output
    if verbose
        println("-" ^ 80)
        println("Final problem in standard form:")
        println("Objective function coefficients (c): ", c)
        println("Constraint matrix (A):\n", Matrix(A))
        println("Right-hand side vector (b): ", b)
        println("Constraint types: ", new_constraint_types)
        println("Variable names: ", vars)
        println("Variable types: ", variable_types)
        println("Variable lower bounds (l): ", l)
        println("Variable upper bounds (u): ", u)
        println("~" ^ 80)
        println("Conversion to standard form completed.")
        println("#" ^ 80)
        println()
    end

    # Return the modified LP problem in standard form
    return LPProblem(
        is_minimize,          # Objective is now minimization
        c,                    # Updated objective function coefficients
        A,                    # Updated constraint matrix
        b,                    # Updated RHS vector
        new_constraint_types, # All constraints are equalities
        l,                    # Updated lower bounds
        u,                    # Updated upper bounds
        vars,                 # Updated variable names
        variable_types        # Updated variable types
    )
end


In [ ]:
"""
    initialize_phase_one(lp_std::LPProblem)

Initializes Phase I by identifying slack and artificial variables and setting up basic and non-basic variable indices.

# Arguments
- `lp_std::LPProblem`: The LP problem in standard form, including slack and artificial variables.

# Returns
- `basic_indices::Vector{Int}`: Indices of basic variables (slack and artificial variables).
- `nonbasic_indices::Vector{Int}`: Indices of non-basic variables.
- `artificial_vars::Vector{Int}`: Indices of artificial variables.
"""
function initialize_phase_one(lp_std::LPProblem)
    m, n = size(lp_std.A)
    # Identify slack variables by their names starting with 's_'
    slack_vars = [i for i in 1:length(lp_std.vars) if startswith(lp_std.vars[i], "s_")]
    
    # Identify artificial variables by their names starting with 'a_'
    artificial_vars = [i for i in 1:length(lp_std.vars) if startswith(lp_std.vars[i], "a_")]

    # Basic indices consist of all slack variables and artificial variables
    basic_indices = vcat(slack_vars, artificial_vars)
    
    # Non-basic indices are the remaining variables
    nonbasic_indices = setdiff(1:n, basic_indices)

    return basic_indices, nonbasic_indices, artificial_vars
end


In [ ]:
"""
    perform_phase_one(lp_std::LPProblem, basic_indices::Vector{Int}, nonbasic_indices::Vector{Int},
                     artificial_vars::Vector{Int}; verbose::Bool=false, tol_feas::Float64=1e-8,
                     max_iterations::Int=1000)

Performs Phase I of the Revised Simplex Method to find an initial basic feasible solution.

# Arguments
- `lp_std::LPProblem`: The LP problem in standard form.
- `basic_indices::Vector{Int}`: Initial basic variable indices (slack and artificial variables).
- `nonbasic_indices::Vector{Int}`: Initial non-basic variable indices.
- `artificial_vars::Vector{Int}`: Indices of artificial variables.
- `verbose::Bool`: Flag for verbose output.
- `tol_feas::Float64`: Tolerance for feasibility.
- `max_iterations::Int`: Maximum number of iterations.

# Returns
- `status::Symbol`: Status after Phase I (:optimal, :infeasible, etc.).
- `basic_indices::Vector{Int}`: Updated basic variable indices.
- `nonbasic_indices::Vector{Int}`: Updated non-basic variable indices.
- `x_B::Vector{Float64}`: Basic variable values.
- `iterations::Int`: Number of iterations performed.
"""
function perform_phase_one(lp_std::LPProblem, basic_indices::Vector{Int}, nonbasic_indices::Vector{Int},
                         artificial_vars::Vector{Int}; verbose::Bool=false, tol_feas::Float64=1e-8,
                         max_iterations::Int=1000)
    m, n = size(lp_std.A)
    iteration = 0
    status = :optimal

    # Objective function for Phase I: minimize sum of artificial variables
    c_phase1 = zeros(n)
    for a_var in artificial_vars
        c_phase1[a_var] = 1.0
    end

    # Initial LU factorization
    B = lp_std.A[:, basic_indices]
    lu_B = try
        lu(Matrix(B))
    catch e
        error("LU factorization failed during Phase I: $(e.msg)")
    end

    x_B = lu_B \ lp_std.b
    x_N = zeros(length(nonbasic_indices))

    if verbose
        println("Phase I: Starting to find initial feasible solution...")
        println("Initial Basic Variables (indices): ", basic_indices)
        println("Initial Non-Basic Variables (indices): ", nonbasic_indices)
        println("Initial Basic Solution x_B: ", x_B)
        println("-" ^ 80)
    end

    while iteration < max_iterations
        iteration += 1

        # Compute dual variables
        y = lu_B' \ c_phase1[basic_indices]

        # Compute reduced costs
        N = lp_std.A[:, nonbasic_indices]
        reduced_costs = c_phase1[nonbasic_indices] - N' * y

        # Check for optimality
        if all(reduced_costs .>= -tol_feas)
            break
        end

        # Determine entering variable using Bland's Rule to prevent cycling
        entering_candidates = findall(reduced_costs .< -tol_feas)
        if isempty(entering_candidates)
            break
        end
        entering_index_in_N = minimum(entering_candidates)
        entering_var = nonbasic_indices[entering_index_in_N]

        # Compute direction d
        d = lu_B \ lp_std.A[:, entering_var]

        # Determine leaving variable using minimum ratio test
        positive_d_indices = findall(d .> tol_feas)
        if isempty(positive_d_indices)
            status = :unbounded
            break
        end

        ratios = x_B[positive_d_indices] ./ d[positive_d_indices]
        min_ratio, pos = findmin(ratios)
        leaving_index_in_B = positive_d_indices[pos]
        leaving_var = basic_indices[leaving_index_in_B]

        # Update basic and non-basic indices
        basic_indices[leaving_index_in_B] = entering_var
        nonbasic_indices[entering_index_in_N] = leaving_var

        # Update B and LU factorization
        B = lp_std.A[:, basic_indices]
        try
            lu_B = lu(Matrix(B))
        catch e
            error("LU factorization failed during Phase I iteration $(iteration): $(e.msg)")
        end

        # Update basic solution
        x_B = lu_B \ lp_std.b
        x_N = zeros(length(nonbasic_indices))

        if verbose
            println("Phase I Iteration: ", iteration)
            println("Entering Variable: ", entering_var, " (", lp_std.vars[entering_var], ")")
            println("Leaving Variable: ", leaving_var, " (", lp_std.vars[leaving_var], ")")
            println("Basic Indices: ", basic_indices)
            println("Basic Solution x_B: ", x_B)
            println("-" ^ 80)
        end
    end

    # Compute the value of the Phase I objective function
    phase1_objective = sum(x_B[findall(in.(basic_indices, artificial_vars))])

    if phase1_objective > tol_feas
        status = :infeasible
    end

    return status, basic_indices, nonbasic_indices, x_B, iteration
end


In [ ]:
"""
    perform_phase_two(lp_phase2::LPProblem, basic_indices::Vector{Int}, nonbasic_indices::Vector{Int},
                     c_phase2::Vector{Float64}; verbose::Bool=false, tol_opt::Float64=1e-10,
                     max_iterations::Int=1000)

Performs Phase II of the Revised Simplex Method to optimize the original objective function.

# Arguments
- `lp_phase2::LPProblem`: The LP problem in standard form after removing artificial variables.
- `basic_indices::Vector{Int}`: Current basic variable indices.
- `nonbasic_indices::Vector{Int}`: Current non-basic variable indices.
- `c_phase2::Vector{Float64}`: Original objective coefficients.
- `verbose::Bool`: Flag for verbose output.
- `tol_opt::Float64`: Tolerance for optimality.
- `max_iterations::Int`: Maximum number of iterations.

# Returns
- `status::Symbol`: Status after Phase II (:optimal, :unbounded, etc.).
- `basic_indices::Vector{Int}`: Updated basic variable indices.
- `nonbasic_indices::Vector{Int}`: Updated non-basic variable indices.
- `x_B::Vector{Float64}`: Basic variable values.
- `iterations::Int`: Number of iterations performed.
"""
function perform_phase_two(lp_phase2::LPProblem, basic_indices::Vector{Int}, nonbasic_indices::Vector{Int},
                         c_phase2::Vector{Float64}; verbose::Bool=false, tol_opt::Float64=1e-10,
                         max_iterations::Int=1000)
    m, n = size(lp_phase2.A)
    iteration = 0
    status = :optimal

    # Initial LU factorization
    B = lp_phase2.A[:, basic_indices]
    lu_B = try
        lu(Matrix(B))
    catch e
        error("LU factorization failed during Phase II: $(e.msg)")
    end

    x_B = lu_B \ lp_phase2.b
    x_N = zeros(length(nonbasic_indices))

    if verbose
        println("Phase II: Starting optimization of original objective function...")
        println("Initial Basic Variables (indices): ", basic_indices)
        println("Initial Non-Basic Variables (indices): ", nonbasic_indices)
        println("Initial Basic Solution x_B: ", x_B)
        println("-" ^ 80)
    end

    while iteration < max_iterations
        iteration += 1

        # Compute dual variables
        y = lu_B' \ c_phase2[basic_indices]

        # Compute reduced costs
        N = lp_phase2.A[:, nonbasic_indices]
        reduced_costs = c_phase2[nonbasic_indices] - N' * y

        # Check for optimality
        if all(reduced_costs .>= -tol_opt)
            break
        end

        # Determine entering variable using Bland's Rule to prevent cycling
        entering_candidates = findall(reduced_costs .< -tol_opt)
        if isempty(entering_candidates)
            break
        end
        entering_index_in_N = minimum(entering_candidates)
        entering_var = nonbasic_indices[entering_index_in_N]

        # Compute direction d
        d = lu_B \ lp_phase2.A[:, entering_var]

        # Determine leaving variable using minimum ratio test
        positive_d_indices = findall(d .> tol_opt)
        if isempty(positive_d_indices)
            status = :unbounded
            break
        end

        ratios = x_B[positive_d_indices] ./ d[positive_d_indices]
        min_ratio, pos = findmin(ratios)
        leaving_index_in_B = positive_d_indices[pos]
        leaving_var = basic_indices[leaving_index_in_B]

        # Update basic and non-basic indices
        basic_indices[leaving_index_in_B] = entering_var
        nonbasic_indices[entering_index_in_N] = leaving_var

        # Update B and LU factorization
        B = lp_phase2.A[:, basic_indices]
        try
            lu_B = lu(Matrix(B))
        catch e
            error("LU factorization failed during Phase II iteration $(iteration): $(e.msg)")
        end

        # Update basic solution
        x_B = lu_B \ lp_phase2.b
        x_N = zeros(length(nonbasic_indices))

        if verbose
            println("Phase II Iteration: ", iteration)
            println("Entering Variable: ", entering_var, " (", lp_phase2.vars[entering_var], ")")
            println("Leaving Variable: ", leaving_var, " (", lp_phase2.vars[leaving_var], ")")
            println("Basic Indices: ", basic_indices)
            println("Basic Solution x_B: ", x_B)
            println("-" ^ 80)
        end
    end

    # Compute the value of the Phase II objective function
    phase2_objective = c_phase2[basic_indices]' * x_B

    # Check for optimality
    reduced_costs = c_phase2[nonbasic_indices] - (lp_phase2.A[:, nonbasic_indices]' * (lu_B' \ c_phase2[basic_indices]))

    if any(reduced_costs .< -tol_opt)
        status = :unbounded
    end

    return status, basic_indices, nonbasic_indices, x_B, iteration
end


In [ ]:
"""
    assemble_solution(lp_phase2::LPProblem, basic_indices::Vector{Int}, nonbasic_indices::Vector{Int},
                     x_B::Vector{Float64})::Dict{String, Float64}

Assembles the full solution vector from basic and non-basic variables.

# Arguments
- `lp_phase2::LPProblem`: The LP problem in standard form after removing artificial variables.
- `basic_indices::Vector{Int}`: Indices of basic variables.
- `nonbasic_indices::Vector{Int}`: Indices of non-basic variables.
- `x_B::Vector{Float64}`: Values of basic variables.

# Returns
- `solution::Dict{String, Float64}`: Mapping of variable names to their values.
"""
function assemble_solution(lp_phase2::LPProblem, basic_indices::Vector{Int}, nonbasic_indices::Vector{Int},
                          x_B::Vector{Float64})::Dict{String, Float64}
    n_original = length(lp_phase2.c)  # Number of original variables (excluding slack and artificial)
    solution = Dict{String, Float64}()

    # Initialize all original variables to zero
    for j in 1:n_original
        solution[lp_phase2.vars[j]] = 0.0
    end

    # Assign values to basic original variables
    for (i, var_idx) in enumerate(basic_indices)
        if var_idx <= n_original
            solution[lp_phase2.vars[var_idx]] = x_B[i]
        end
    end

    # Non-basic original variables remain zero

    return solution
end


In [ ]:
"""
    revised_simplex_method(lp::LPProblem; verbose::Bool=false,
                           tol_opt::Float64=1e-10, tol_feas::Float64=1e-8,
                           max_iterations::Int=1000)::SimplexResult

Solves the linear programming problem defined by `lp` using the Revised Simplex Method with Phase I and Phase II.

# Arguments
- `lp::LPProblem`: The linear programming problem.
- `verbose::Bool`: If `true`, prints detailed iteration information.
- `tol_opt::Float64`: Tolerance for optimality conditions.
- `tol_feas::Float64`: Tolerance for feasibility checks.
- `max_iterations::Int`: Maximum number of iterations for each phase.

# Returns
- `SimplexResult`: A structure containing the status, solution vector, optimal objective value, and number of iterations.
"""
function revised_simplex_method_test(lp::LPProblem; verbose::Bool=false,
                               tol_opt::Float64=1e-10, tol_feas::Float64=1e-8,
                               max_iterations::Int=1000)::SimplexResult
    # Step 1: Convert LP to standard form
    lp_std = convert_to_standard_form_test(lp; verbose=verbose)
    m, n = size(lp_std.A)

    # Step 2: Initialize Phase I
    basic_indices, nonbasic_indices, artificial_vars = initialize_phase_one(lp_std)

    if verbose
        println("Phase I: Initializing...")
        println("Initial Basic Variables (indices): ", basic_indices)
        println("Initial Non-Basic Variables (indices): ", nonbasic_indices)
        println("Artificial Variables: ", artificial_vars)
        println("-"^80)
    end

    # Step 3: Perform Phase I
    status_phase1, basic_indices, nonbasic_indices, x_B, iterations_phase1 = perform_phase_one(
        lp_std, basic_indices, nonbasic_indices, artificial_vars;
        verbose=verbose, tol_feas=tol_feas, max_iterations=max_iterations
    )

    if verbose
        println("Phase I completed in $(iterations_phase1) iterations with status: $(status_phase1)")
        println("-"^80)
    end

    if status_phase1 == :infeasible
        if verbose
            println("Original LP problem is infeasible.")
            println("#"^80)
            println()
        end
        return SimplexResult(:infeasible, Dict{String, Float64}(), 0.0, iterations_phase1)
    end

    # Step 4: Prepare for Phase II by removing artificial variables
    # Identify indices of artificial variables
    artificial_vars = [i for i in 1:length(lp_std.vars) if startswith(lp_std.vars[i], "a_")]

    # Remove artificial variables from the problem
    remaining_vars_indices = setdiff(1:size(lp_std.A, 2), artificial_vars)
    A_phase2 = lp_std.A[:, remaining_vars_indices]
    c_phase2 = lp_std.c[remaining_vars_indices]
    vars_phase2 = [lp_std.vars[j] for j in remaining_vars_indices]
    variable_types_phase2 = [lp_std.variable_types[j] for j in remaining_vars_indices]
    l_phase2 = [lp_std.l[j] for j in remaining_vars_indices]
    u_phase2 = [lp_std.u[j] for j in remaining_vars_indices]
    n_phase2 = length(vars_phase2)
    m_phase2 = size(A_phase2, 1)

    # Update basic and non-basic indices after removing artificial variables
    basic_indices = setdiff(basic_indices, artificial_vars)
    nonbasic_indices = setdiff(1:size(A_phase2, 2), basic_indices)

    # Create a new LPProblem for Phase II using positional arguments
    lp_phase2 = LPProblem(
        lp_std.is_minimize,            # is_minimize
        c_phase2,                      # c
        A_phase2,                      # A
        lp_std.b,                      # b
        fill('E', m_phase2),           # constraint_types
        l_phase2,                      # l
        u_phase2,                      # u
        vars_phase2,                   # vars
        variable_types_phase2          # variable_types
    )

    if verbose
        println("Phase II: Removing artificial variables and setting up original objective.")
        println("Updated Basic Variables (indices): ", basic_indices)
        println("Updated Non-Basic Variables (indices): ", nonbasic_indices)
        println("-"^80)
    end

    # Step 5: Perform Phase II
    status_phase2, basic_indices, nonbasic_indices, x_B, iterations_phase2 = perform_phase_two(
        lp_phase2, basic_indices, nonbasic_indices, lp_phase2.c;
        verbose=verbose, tol_opt=tol_opt, max_iterations=max_iterations
    )

    if verbose
        println("Phase II completed in $(iterations_phase2) iterations with status: $(status_phase2)")
        println("-"^80)
    end

    if status_phase2 == :unbounded
        if verbose
            println("The LP problem is unbounded.")
            println("#"^80)
            println()
        end
        return SimplexResult(:unbounded, Dict{String, Float64}(), 0.0, iterations_phase1 + iterations_phase2)
    end

    # Step 6: Assemble the final solution
    solution = assemble_solution(lp_phase2, basic_indices, nonbasic_indices, x_B)
    optimal_value = lp_phase2.c' * [solution[var] for var in lp_phase2.vars]

    if verbose
        println("Optimal Solution:")
        for (var, val) in sort(collect(solution), by=x->x[1])
            println("  $(var) = $(val)")
        end
        println("Optimal Objective Value: $(optimal_value)")
        println("Revised Simplex Method completed successfully.")
        println("#"^80)
        println()
    end

    total_iterations = iterations_phase1 + iterations_phase2
    return SimplexResult(:optimal, solution, optimal_value, total_iterations)
end


In [24]:
# # Example Usage

# # Define a sample LPProblem
# # Maximize: 3x1 + 2x2
# # Subject to:
# #     x1 + 2x2 ≤ 8
# #     4x1       ≥ 16
# #          4x2 ≤ 12
# #     x1, x2 ≥ 0

# using SparseArrays

# # Define the LP components
# c = [3.0, 2.0]
# A_dense = [
#     1.0  2.0;
#     4.0  0.0;
#     0.0  4.0
# ]
# A = sparse(A_dense)
# b = [8.0, 16.0, 12.0]
# constraint_types = ['L', 'G', 'L']
# l = [0.0, 0.0]
# u = [Inf, Inf]
# vars = ["x1", "x2"]
# variable_types = [:Continuous, :Continuous]
# is_minimize = false  # Original problem is a maximization

# # Create the LPProblem instance
# lp = LPProblem(
#     is_minimize,
#     c,
#     A,
#     b,
#     constraint_types,
#     l,
#     u,
#     vars,
#     variable_types
# )

# # Solve the LP using Revised Simplex Method
# result = revised_simplex_method_test(lp; verbose=true)

# # Output the results
# println("Solution Status: ", result.status)
# println("Number of Iterations: ", result.iterations)
# println("Optimal Objective Value: ", result.objective)
# println("Optimal Solution:")
# for (var, val) in sort(collect(result.solution), by=x->x[1])
#     println("  $(var) = $(val)")
# end
